# Unsaturated poroelasticity: Convergence Test #1

* Author: Jhabriel Varela
* E-mail: jhabriel.varela@uib.no
* Date: 12.04.2019
* Institution: PMG - UiB - Norway

## Description of the problem

In this noteboook, we present a convergence test of the unsaturated Biot's equations in PorePy. The equation is discretized using MPFA [[1](#ref)] for the flow, MPSA  [[2](#ref)] for the mechanics and backward Euler in time. We use the Automatic Differentiation framework from PorePy and solve the non-linear system using a modified Picard Iteration [[3](#ref)].

The complete set of equations reads [[4](#ref)]:

$$
    \nabla \cdot \mathbf{\sigma}_e - \alpha \nabla \left(S p\right) = F , \\
    \mathbf{\sigma}_e = \frac{1}{2} D \left(\nabla \mathbf{u} + \nabla \mathbf{u}^T\right), \\
    \left[(\alpha - n ) C_s S^2 + n C_f S \right] \frac{\partial p}{\partial t} + \left[(\alpha - n) C_s S p + n \right] \frac{\partial S}{\partial t} + \alpha S \frac{\partial}{\partial t}\left(\nabla \cdot \mathbf{u}\right) + \nabla \cdot \mathbf{q} = f, \\
    \mathbf{q} = -\frac{K}{\mu_f} k_r^w \left(\nabla p - \rho_f \mathbf{g}\right),
$$

where $\mathbf{\sigma}_e$ is the effective stress, $\alpha$ is the Biot's coefficient, $S$ and $p$ are the water saturation and pressure, $F$ is the mechanics source term, $D$ is the stiffness matrix, $\mathbf{u}$ is the displacement vector, $n$ is the initial porosity, $C_s$ and $C_f$ are the solid and water compressibilities, $\mathbf{q}$ is the water Darcy's velocity, $f$ is the flow source term, $K$ is the permeability tensor, $\mu_f$ and $\rho_f$ are the water viscosity and density and $\mathbf{g}$ is the acceleration vector.

For simplicity, in this example we neglect gravity contributions, i.e.: $\mathbf{g} = 0$. Moreover, we employ $\lambda_s = \mu_s = \mu_f = C_f = C_s =  K = \alpha = 1$ and $n = 0.5$.

In order to keep things simple, we use the following non-linear relationships to describe the dependecy of the saturation and relative permeability on the pressure [[5](#ref)]:
$$
S(p) = \frac{1}{1-p}, \\
k_r^w(p) = p^2.
$$

Our aim is to solve the above set of equations on a unit square, assuming that an exact displacement and pressure distributions are known. For this purpose, we choose:

$$
\mathbf{u}(x,y,t) = \begin{pmatrix} tx(x-1)y(y-1) \\ -tx(x-1)y(y-1) \end{pmatrix}\\
p(x,y,t) = -tx(1-x)y(1-y) - 1.
$$

If we only use Dirichlet boundary conditions, the following identities hold true

$$
\mathbf{u}(0,y,t) = \mathbf{u}(1,y,t) = \mathbf{u}(x,0,t) = \mathbf{u}(x,1,t) = \mathbf{u}(x,y,0) = (0,0), \\
p(0,y,t) = p(1,y,t) = p(x,0,t) = p(x,1,t) = p(x,y,0) = -1.
$$

With the prescribed solutions (and with the help of a symbolic software) we can derive the expressions for the source terms $F$ and $f$.

## Measuring the errors

We measure the errors and the convergence rate of the primary variables using the $L^2-$norm:

$$
\epsilon_{p} = ||p - \tilde{p}||, \qquad \epsilon_{\mathbf{u}} = ||\mathbf{u} - \mathbf{\tilde{u}}||,
$$

where the variables with tilde denote the numerical solutions and the variables without them, the analytical solutions.

We analyse the combinations of four different spatial and time refinements, i.e.: the pairs of grid and time step sizes: $(0.1,0.1)$, $(0.05,0.05)$, $(0.025,0.025)$, and $(0.0125,0.0125)$. The final simulation time for all cases is 1.

## Importing modules

In [1]:
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as plt
import scipy.optimize as opt
import porepy as pp
from porepy.ad.forward_mode import Ad_array
np.set_printoptions(precision=4, suppress = True)

## Convergence analysis function

In [2]:
def unsat_poro_conv_test(cells_num,timeLevels_num):

    ## Functions

    ### Source term: Mechanics

    def source_mech(g,t):

        # x and  y cell centers
        x = g.cell_centers[0]
        y = g.cell_centers[1]

        # Initializing mechanics source term
        F = np.zeros(g.dim * g.num_cells)

        F[::2] = (t*x*y*(y - 1) + t*y*(x - 1)*(y - 1))/(t*x*y*(x - 1)*(y - 1) + 2) + \
                2*t*x*(x - 1) - \
                2*t*x*(y - 1) - \
                2*t*y*(x - 1) + \
                6*t*y*(y - 1) - \
                ((t*x*y*(x - 1)*(y - 1) + 1)*(t*x*y*(y - 1) + t*y*(x - 1)*(y - 1)))/(t*x*y*(x - 1)*(y - 1) + 2)**2 - \
                2*t*(x - 1)*(y - 1) - \
                2*t*x*y        
        
        # y-component of the mechanics source term
        
        F[1::2] = (t*x*y*(x - 1) + t*x*(x - 1)*(y - 1))/(t*x*y*(x - 1)*(y - 1) + 2) - \
                  6*t*x*(x - 1) + \
                  2*t*x*(y - 1) + \
                  2*t*y*(x - 1) - \
                  2*t*y*(y - 1) - \
                  ((t*x*y*(x - 1)*(y - 1) + 1)*(t*x*y*(x - 1) + t*x*(x - 1)*(y - 1)))/(t*x*y*(x - 1)*(y - 1) + 2)**2 + \
                  2*t*(x - 1)*(y - 1) + \
                  2*t*x*y
        
        return F

    ### Source term: Flow

    def source_flow(g,t):

        # x and  y cell centers
        x = g.cell_centers[0]
        y = g.cell_centers[1]

        # Flow source term
        f = 2*(t*x*y*(x - 1)*(y - 1) + 1)*(t*x*y*(x - 1) + t*x*(x - 1)*(y - 1))**2 \
            - (x*y*(x - 1) - x*y*(y - 1) + x*(x - 1)*(y - 1) - y*(x - 1)*(y - 1))/(t*x*y*(x - 1)*(y - 1) + 2) \
            + 2*(t*x*y*(x - 1)*(y - 1) + 1)*(t*x*y*(y - 1) + t*y*(x - 1)*(y - 1))**2 \
            + 2*t*x*(t*x*y*(x - 1)*(y - 1) + 1)**2*(x - 1) \
            + 2*t*y*(t*x*y*(x - 1)*(y - 1) + 1)**2*(y - 1) \
            - x*y*(x - 1)*(y - 1)*(1/(2*(t*x*y*(x - 1)*(y - 1) + 2)) + 1/(2*(t*x*y*(x - 1)*(y - 1) + 2)**2) - ((t*x*y*(x - 1)*(y - 1) + 1)/(2*(t*x*y*(x - 1)*(y - 1) + 2)) - 1/2)/(t*x*y*(x - 1)*(y - 1) + 2)**2)
    
        return f

    ### Analytical solution

    def analytical(g,t):

        x_cntr = g.cell_centers[0]
        y_cntr = g.cell_centers[1]

        u = np.zeros(g.dim * g.num_cells)

        u[::2] = t * (1-x_cntr) * x_cntr * (1-y_cntr) * y_cntr
        u[1::2] = - u[::2]

        p = -t * (1-x_cntr) * x_cntr * (1-y_cntr) * y_cntr - 1

        return u,p


    ### Computation of the arithmetic averaged relative permeabilities

    def arithmetic_mpfa_hyd(krw,g,bc,bc_val,h_m0):

        """
        Computes the arithmetic average of the relative permability

        SYNOPSIS:
            arithmetic_mpfa_hyd(krw,g,bc_flow,bc_val_flow,h_m0)

        INPUT ARGUMENTS:
            krw              - Lambda function, relative permeability function krw = f(psi)
            g                - PorePy grid object
            bc_flow          - PorePy scalar boundary object
            bc_val_flow      - NumPy array, containing values of boundary conditions
            h_m0             - NumPy array, containing values of hydraulic head at the cell centers

        RETURNS:
            krw_ar           - Numpy array, contatining arithmetic averaged relative permeabilities 
                          at the face centers
        """

        z_cntr = g.cell_centers[2,:]     # z-values of cell centers
        z_fcs = g.face_centers[2,:]      # z-values of face centers 
        neu_fcs = bc.is_neu.nonzero()    # neumann boundary faces
        dir_fcs = bc.is_dir.nonzero()    # dirichlet boundary faces
        int_fcs = g.get_internal_faces() # internal faces

        fcs_neigh = np.zeros((g.num_faces,2),dtype=int) #          
        fcs_neigh[:,0] = g.cell_face_as_dense()[0]      # faces neighbouring mapping
        fcs_neigh[:,1] = g.cell_face_as_dense()[1]      # 

        int_fcs_neigh = fcs_neigh[int_fcs]              # internal faces neighbouring mapping

        # Initializing 
        krw_ar = np.zeros(g.num_faces)

        # Neumann boundaries relative permeabilities
        krw_ar[neu_fcs] = 1.

        # Dirichlet boundaries relative permeabilities
        dir_cells_neigh = fcs_neigh[dir_fcs] # neighboring cells of dirichlet faces
        dir_cells = dir_cells_neigh[(dir_cells_neigh >= 0).nonzero()] # cells that share a dirichlet face 
        krw_ar[dir_fcs] = 0.5 * (krw(bc_val[dir_fcs]-z_fcs[dir_fcs]) + 
                                 krw(h_m0[dir_cells]-z_cntr[dir_cells]))

        # Internal faces relative permeabilities
        krw_ar[int_fcs] = 0.5 * (krw(h_m0[int_fcs_neigh[:,0]] - z_cntr[int_fcs_neigh[:,0]]) +
                                 krw(h_m0[int_fcs_neigh[:,1]] - z_cntr[int_fcs_neigh[:,1]]))

        return krw_ar

    ## Setting-up the grid

    Nx = Ny = cells_num
    Lx = Ly = 1
    g = pp.CartGrid([Nx,Ny], [Lx,Ly])
    g.compute_geometry()
    V = g.cell_volumes
    d = dict() # initializing data dictionary
    
    ## Physical parameters
    mu_s = 1
    lambda_s = 1
    mu_f = 1
    C_s = 1
    C_f = 1
    K = 1
    alpha = 1
    n = 0.5

    ## Creating the second and fourth order tensors

    perm = pp.SecondOrderTensor(g.dim, K*np.ones(g.num_cells))
    constit = pp.FourthOrderTensor(g.dim, mu_s*np.ones(g.num_cells), lambda_s*np.ones(g.num_cells))

    ## Boundary conditions

    ## Boundary and initial conditions
    b_faces = g.tags['domain_boundary_faces'].nonzero()[0]

    # Extracting indices of boundary faces w.r.t g
    x_min = b_faces[g.face_centers[0,b_faces] < 0.0001]
    x_max = b_faces[g.face_centers[0,b_faces] > 0.9999*Lx]
    y_min = b_faces[g.face_centers[1,b_faces] < 0.0001]
    y_max = b_faces[g.face_centers[1,b_faces] > 0.9999*Ly]

    # Extracting indices of boundary faces w.r.t b_faces
    west   = np.in1d(b_faces,x_min).nonzero()
    east   = np.in1d(b_faces,x_max).nonzero()
    south  = np.in1d(b_faces,y_min).nonzero()
    north  = np.in1d(b_faces,y_max).nonzero()

    ### Flow Boundary conditions

    # Setting the tags at each boundary side
    labels_flow = np.array([None]*b_faces.size)
    labels_flow[east]   = 'dir'
    labels_flow[west]   = 'dir'
    labels_flow[south]  = 'dir'
    labels_flow[north]  = 'dir'

    # Constructing the bc object
    bc_flow = pp.BoundaryCondition(g, b_faces, labels_flow)

    # Constructing the boundary values array
    bc_val_flow = -1 * np.ones(g.num_faces)

    ### Elasticity Boundary conditions

    # Setting the tags at each boundary side
    labels_mech = np.array([None]*b_faces.size)
    labels_mech[east]   = 'dir'
    labels_mech[west]   = 'dir'
    labels_mech[south]  = 'dir'
    labels_mech[north]  = 'dir'

    # Constructing the bc object
    bc_mech = pp.BoundaryConditionVectorial(g, b_faces, labels_mech)

    # Constructing the boundary values array
    bc_val_mech = np.zeros(g.num_faces * g.dim)

    ## Creating data objects

    # Mechanics data
    specified_parameters_mech = {"fourth_order_tensor": constit, 
                                 "bc": bc_mech, 
                                 "biot_alpha" : 1.,
                                 "bc_values": bc_val_mech}

    d = pp.initialize_default_data(g,d,"mechanics", specified_parameters_mech)

    # Flow data
    specified_parameters_flow = {"second_order_tensor": perm, 
                                 "bc": bc_flow, 
                                 "biot_alpha": 1.,
                                 "bc_values": bc_val_flow}

    d = pp.initialize_default_data(g,d,"flow", specified_parameters_flow)

    ## Performing MPSA/MPFA discretization

    ### Discretization

    solver_biot = pp.Biot("mechanics","flow")
    solver_biot.discretize(g,d)

    ### Retrieving operators

    biot_F = d['discretization_matrices']['flow']['flux']
    biot_boundF = d['discretization_matrices']['flow']['bound_flux']
    biot_compat = d['discretization_matrices']['flow']['biot_stabilization']
    biot_divF = pp.fvutils.scalar_divergence(g)

    biot_S = d['discretization_matrices']['mechanics']['stress']
    biot_boundS = d['discretization_matrices']['mechanics']['bound_stress']
    biot_divU = d['discretization_matrices']['mechanics']['div_d']
    biot_gradP = d['discretization_matrices']['mechanics']['grad_p']
    biot_boundUCell = d['discretization_matrices']['mechanics']['bound_displacement_cell']
    biot_boundUFace = d['discretization_matrices']['mechanics']['bound_displacement_face']
    biot_boundUPressure = d['discretization_matrices']['mechanics']['bound_displacement_pressure']
    biot_divS = pp.fvutils.vector_divergence(g)

    ### Creating discrete operators

    F      = lambda x: biot_F * x                       #
    boundF = lambda x: biot_boundF * x                  # Flow 
    compat = lambda x: biot_compat * x                  # operators
    divF    = lambda x: biot_divF * x                   #

    S      = lambda x: biot_S * x                       #
    boundS = lambda x: biot_boundS * x                  #  
    divU   = lambda x: biot_divU * x                    # 
    divS   = lambda x: biot_divS * x                    # Mechanics 
    gradP  = lambda x: biot_divS * biot_gradP * x       # operators
    boundUCell = lambda x: biot_boundUCell * x          #
    boundUFace = lambda x : biot_boundUFace * x         #
    boundUPressure = lambda x: biot_boundUPressure * x  # 

    ## Creating AD variables

    u_init = np.zeros(g.dim * g.num_cells)    # initial displacement field
    p_init = -1*np.ones(g.num_cells)          # initial pressure distribution

    u_ad = Ad_array(u_init.copy(), sps.diags(np.ones(g.num_cells * g.dim))) # initializing u_ad
    p_ad = Ad_array(p_init.copy(), sps.diags(np.ones(g.num_cells)))         # initializing p_ad

    # Water retention curves

    # Water content
    Sw = lambda p: 1/(1-p)

    # Specific saturation capacity
    C = lambda p: 1/((1-p)**2)

    # Relative permeability
    krw = lambda p: p**2

    ## Time parameters

    t0 = 0                                # [s] Initial time
    tf = 1                                # [s] Final simulation time
    tLevels = timeLevels_num              # [-] Time levels
    times = np.linspace(t0,tf,tLevels+1)  # [s] Vector of time evaluations
    dt = np.diff(times)                   # [s] Vector of time steps

    ## Discrete equations

    # Arithmetic mean of relative permeability
    krw_ar = lambda p_m: arithmetic_mpfa_hyd(krw,g,bc_flow,bc_val_flow,p_m)

    # Generalized Hooke's law
    T = lambda u: S(u) + boundS(bc_val_mech)

    # Momentum conservation equation (Mechanics contribution)
    u_eq1 = lambda u: divS(T(u)) 

    # Momentum conservation equation  (Flow contribution)
    u_eq2 = lambda p,p_n: gradP(p*Sw(p_n)) - source_mech(g,times[tt])*V[0]

    # Multiphase Darcy's law
    Q = lambda p,p_m: (1/mu_f) * (F(p) + boundF(bc_val_flow)) * krw_ar(p_m)

    # Mass conservation equation (Mechanics contribution)
    p_eq1 = lambda u,u_n,p_n,dt: alpha * (divU(u-u_n)/dt) * Sw(p_n)

    # Mass conservation equation (Flow contribution)
    p_eq2 = lambda p,p_m,p_n,dt:  (compat(p-p_n)/dt) * Sw(p_n)  \
                                  + ((p-p_n)/dt) * ((alpha-n)*C_s*Sw(p_n)**2 + n*C_f*Sw(p_n)) * V  \
                                  + (((p-p_m)*C(p_m) + Sw(p_m) - Sw(p_n))/dt) * ((alpha-n)*C_s*Sw(p_n)*p_n+n) * V \
                                  + divF(Q(p,p_m)) \
                                  - source_flow(g,times[tt]) * V

    ## The time loop

    ## Newton parameters
    newton_param = dict()
    newton_param['max_tol'] = 1E-9            # [cm] maximum absolute tolerance (pressure head)
    newton_param['max_iter'] = 10             # [iter] maximum number of iterations
    newton_param['res_norm'] = 100             # [cm] absolute tolerance
    newton_param['iter'] = 1                  # [iter] iteration

    print("\n Performing simulation with h={} and dt={} \n".format(1/cells_num,1/timeLevels_num))
    
    tt = 0
    while times[tt] < tf:

        u_n = u_ad.val.copy()            # u: current time step
        p_n = p_ad.val.copy();           # p: current time step
        tt += 1

        newton_param.update({'res_norm':1000, 'iter':1})      

        while newton_param['res_norm'] > newton_param['max_tol'] and \
              newton_param['iter'] <= newton_param['max_iter']:

            p_m = p_ad.val.copy()            # current iteration level (m)

            # Calling equations
            eq1 = u_eq1(u_ad)
            eq2 = u_eq2(p_ad,p_n)
            eq3 = p_eq1(u_ad,u_n,p_n,dt[tt-1])
            eq4 = p_eq2(p_ad,p_m,p_n,dt[tt-1])

            # Assembling Jacobian of the coupled system
            J_mech = np.hstack((eq1.jac,eq2.jac)) # Jacobian blocks (mechanics)
            J_flow = np.hstack((eq3.jac,eq4.jac)) # Jacobian blocks (flow)
            J = sps.bmat(np.vstack((J_mech,J_flow)),format='csc') # Jacobian (coupled)

            # Determining residual of the coupled system
            R_mech = eq1.val + eq2.val            # Residual (mechanics)
            R_flow = eq3.val + eq4.val            # Residual (flow)
            R = np.hstack((R_mech,R_flow))        # Residual (coupled)

            y = sps.linalg.spsolve(J,-R)                  # 
            u_ad.val = u_ad.val + y[:g.dim*g.num_cells]   # Newton update
            p_ad.val = p_ad.val + y[g.dim*g.num_cells:]   #

            newton_param['res_norm'] = np.linalg.norm(R)

            if newton_param['res_norm'] <= newton_param['max_tol'] and \
               newton_param['iter'] <= newton_param['max_iter']:
                print('Time: {:.4f} [s] \t Iter: {} \t Error: {:.4e}'.format(times[tt],newton_param['iter'],newton_param['res_norm']))
            elif newton_param['iter'] > newton_param['max_iter']:
                print('Error: Newton method did not converge!')
            else:
                newton_param['iter'] += 1
                
    
    ## Analytical solution
    u_ex,p_ex = analytical(g,times[-1])
    
    ## Computing norms
    e_p = np.linalg.norm(p_ex-p_ad.val)
    e_u = np.linalg.norm(u_ex-u_ad.val)
    
    return e_p, e_u

## Performing test

In [3]:
# Spatial step size
N = np.array([10,20,40,80])
h = 1/N

# Time step size
T = np.array([10,20,40,80])
tau = 1/T

# Extracting the errors
e_p = np.zeros(len(N))
e_u = np.zeros(len(N))
for i in range(len(N)):
    e_p[i],e_u[i] = unsat_poro_conv_test(N[i],T[i])
    
# Computing the error reduction
e_p_red = np.zeros(len(e_p)-1)
e_u_red = np.zeros(len(e_u)-1)
for i in range(len(e_p_red)):
    e_p_red[i] = e_p[i]/e_p[i+1]
    e_u_red[i] = e_u[i]/e_u[i+1]


 Performing simulation with h=0.1 and dt=0.1 

Time: 0.1000 [s] 	 Iter: 4 	 Error: 5.0076e-10
Time: 0.2000 [s] 	 Iter: 5 	 Error: 1.8300e-11
Time: 0.3000 [s] 	 Iter: 5 	 Error: 9.3908e-11
Time: 0.4000 [s] 	 Iter: 5 	 Error: 2.9895e-10
Time: 0.5000 [s] 	 Iter: 5 	 Error: 7.3395e-10
Time: 0.6000 [s] 	 Iter: 6 	 Error: 1.5637e-11
Time: 0.7000 [s] 	 Iter: 6 	 Error: 3.3934e-11
Time: 0.8000 [s] 	 Iter: 6 	 Error: 6.6397e-11
Time: 0.9000 [s] 	 Iter: 6 	 Error: 1.2005e-10
Time: 1.0000 [s] 	 Iter: 6 	 Error: 2.0393e-10

 Performing simulation with h=0.05 and dt=0.05 

Time: 0.0500 [s] 	 Iter: 4 	 Error: 1.1120e-11
Time: 0.1000 [s] 	 Iter: 4 	 Error: 9.5306e-11
Time: 0.1500 [s] 	 Iter: 4 	 Error: 3.2648e-10
Time: 0.2000 [s] 	 Iter: 4 	 Error: 7.7897e-10
Time: 0.2500 [s] 	 Iter: 5 	 Error: 6.6230e-12
Time: 0.3000 [s] 	 Iter: 5 	 Error: 1.3785e-11
Time: 0.3500 [s] 	 Iter: 5 	 Error: 2.5619e-11
Time: 0.4000 [s] 	 Iter: 5 	 Error: 4.3829e-11
Time: 0.4500 [s] 	 Iter: 5 	 Error: 7.0387e-11
Time: 0.5

## Showing results

### Pressure Error

In [4]:
print('Spatial step size:',h)
print('Time step size   :',tau)
print('Pressure error   :',e_p)
print('Error reduction  :',np.concatenate(([np.NaN],e_p_red)))

Spatial step size: [0.1    0.05   0.025  0.0125]
Time step size   : [0.1    0.05   0.025  0.0125]
Pressure error   : [0.004  0.002  0.001  0.0005]
Error reduction  : [   nan 2.0087 2.0194 2.0364]


### Displacement error

In [5]:
print('Spatial step size  :',h)
print('Time step size     :',tau)
print('Displacement error :',e_u)
print('Error reduction    :',np.concatenate(([np.NaN],e_u_red)))

Spatial step size  : [0.1    0.05   0.025  0.0125]
Time step size     : [0.1    0.05   0.025  0.0125]
Displacement error : [0.0089 0.0045 0.0023 0.0011]
Error reduction    : [   nan 1.9797 1.9931 1.9812]


## References
<a id='ref'></a>

[1]: *Aavatsmark, I. (2002). An introduction to multipoint flux approximations for quadrilateral grids. Computational Geosciences, 6(3-4), 405-432.*

[2]: *Keilegavlen, Eirik, and Jan Martin Nordbotten. "Finite volume methods for elasticity with weak symmetry." International Journal for Numerical Methods in Engineering 112.8 (2017): 939-962.*

[3]: *Celia, M. A., Bouloutas, E. T., & Zarba, R. L. (1990). A general mass‐conservative numerical solution for the unsaturated flow equation. Water resources research, 26(7), 1483-1496.*

[4]: *Schrefler, B. A. (1987). The finite element method in the deformation and consolidation of porous media. Wiley.*

[5]: *Radu, F. A., & Wang, W. (2014). Convergence analysis for a mixed finite element scheme for flow in strictly unsaturated porous media. Nonlinear Analysis: Real World Applications, 15, 266-275.*